In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
tf.VERSION

'1.15.0'

In [3]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.layers.advanced_activations import LeakyReLU
from functools import partial
from keras.layers.merge import _Merge
import keras.backend as K
import numpy as np

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
optimizer = Adam(0.0002, 0.5)
num_classes = 10

def Generator():
  model = Sequential()
  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))
  model.summary()

  noise = Input(shape=(latent_dim,))
  label = Input(shape=(1,), dtype='int32')
  label_embedding = Flatten()(Embedding(num_classes, latent_dim)(label))

  model_input = multiply([noise, label_embedding])
  img = model(model_input)

  return Model(inputs = [noise, label], outputs = img)

def Discriminator():
  model = Sequential()
  model.add(Dense(512, input_dim=np.prod(img_shape)))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))
  model.summary()

  img = Input(shape=img_shape)
  label = Input(shape=(1,), dtype='int32')
  label_embedding = Flatten()(Embedding(num_classes, np.prod(img_shape))(label))
  flat_img = Flatten()(img)
  model_input = multiply([flat_img, label_embedding])
  validity = model(model_input)

  return Model(inputs = [img, label], outputs = validity)


# Build and compile the discriminator
D = Discriminator()
D.compile(loss=['binary_crossentropy'], optimizer=optimizer, metrics=['accuracy'])

# Build the generator
G = Generator()

# The generator takes noise and the target label as input
# and generates the corresponding digit of that label
noise = Input(shape=(latent_dim,))
label = Input(shape=(1,))
img = G([noise, label])

# For the combined model we will only train the generator
D.trainable = False

# The discriminator takes generated image as input and determines validity
# and the label of that image
valid = D([img, label])

# The combined model  (stacked generator and discriminator)
# Trains generator to fool discriminator
combined = Model([noise, label], valid)
combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)              

In [5]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import os
import numpy as np

os.chdir('/content/drive/My Drive/GitHub Repositories/Generative Models')
baseDir = './Conditional Generative Adversarial Nets'
outputDir = os.path.join(baseDir, 'outputs')
if not os.path.exists(outputDir):
  os.makedirs(outputDir)
  print('Output Directory Created to save Results')

epochs=20000
batch_size=32
sample_interval=50

def save_imgs(epoch):
  r, c = 2, 5
  noise = np.random.normal(0, 1, (r * c, 100))
  sampled_labels = np.arange(0, 10).reshape(-1, 1)

  gen_imgs = G.predict([noise, sampled_labels])

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
          axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
          axs[i,j].axis('off')
          cnt += 1
  fig.savefig(outputDir+'/image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()
  plt.close()

# Load the dataset
(X_train, y_train), (_, _) = mnist.load_data()

# Configure input
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)
y_train = y_train.reshape(-1, 1)

# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):
  epoch+=1
  # ---------------------
  #  Train Discriminator
  # ---------------------

  # Select a random half batch of images
  idx = np.random.randint(0, X_train.shape[0], batch_size)
  imgs, labels = X_train[idx], y_train[idx]

  # Sample noise as generator input
  noise = np.random.normal(0, 1, (batch_size, 100))

  # Generate a half batch of new images
  gen_imgs = G.predict([noise, labels])

  # Train the discriminator
  d_loss_real = D.train_on_batch([imgs, labels], valid)
  d_loss_fake = D.train_on_batch([gen_imgs, labels], fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # ---------------------
  #  Train Generator
  # ---------------------

  # Condition on labels
  sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

  # Train the generator
  g_loss = combined.train_on_batch([noise, sampled_labels], valid)

  # Plot the progress
  print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

  # If at save interval => save generated image samples
  if epoch % sample_interval == 0:
      save_imgs(epoch)

    




Output hidden; open in https://colab.research.google.com to view.

In [0]:
import imageio
import glob
images = []
for file_name in os.listdir(outputDir):
    if file_name.endswith('.png'):
        file_path = os.path.join(outputDir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave(os.path.join(baseDir, 'CGAN-training-outputs.gif'), images, fps = 5)